In [5]:
from bs4 import BeautifulSoup as bs
import requests
page = requests.get("https://sofifa.com/players?offset=0")

In [21]:
soup = bs(page.content, 'lxml')

In [ ]:
print(soup.prettify())

In [63]:
t=soup.find('table', {'class': 'table table-hover persist-area'}).find('tbody').find_all('a')
for i in t:
    print(i['href'],i.text)
    #print(i)

/players?na=52 
/player/158023/lionel-messi/ L. Messi
/players?pn=21 CF
/players?pn=23 RW
/players?pn=25 ST
/team/241/fc-barcelona/ FC Barcelona
/players?na=38 
/player/20801/c-ronaldo-dos-santos-aveiro/ Cristiano Ronaldo
/players?pn=25 ST
/players?pn=27 LW
/team/45/juventus/ Juventus
/players?na=54 
/player/190871/neymar-da-silva-santos-jr/ Neymar Jr
/players?pn=27 LW
/players?pn=18 CAM
/team/73/paris-saint-germain/ Paris Saint-Germain
/players?na=45 
/player/193080/david-de-gea-quintana/ De Gea
/players?pn=0 GK
/team/11/manchester-united/ Manchester United
/players?na=7 
/player/192985/kevin-de-bruyne/ K. De Bruyne
/players?pn=18 CAM
/players?pn=14 CM
/team/10/manchester-city/ Manchester City
/players?na=7 
/player/183277/eden-hazard/ E. Hazard
/players?pn=27 LW
/players?pn=21 CF
/team/5/chelsea/ Chelsea
/players?na=10 
/player/177003/luka-modric/ L. Modrić
/players?pn=14 CM
/team/243/real-madrid/ Real Madrid
/players?na=60 
/player/176580/luis-suarez/ L. Suárez
/players?pn=25 ST
/te

In [24]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.Tag]